In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2

from google.colab import drive
drive.mount('/content/drive')
# !ls /content/drive/Othercomputers/My\ Laptop/Thesis/ab_ucf50

DATASET_PATH = '/content/drive/Othercomputers/My Laptop/Thesis/ab_ucf50'
ALL_CLASSES = os.listdir(DATASET_PATH)
# ALL_CLASSES = ALL_CLASSES[:10]
ACTION_BANK_ROWS = 205
ACTION_BANK_COLS = 73

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
def create_dataset():
  data_list = []
  labels_list = []

  for i, category in enumerate(ALL_CLASSES, start=0):
    category_files = os.listdir(DATASET_PATH + '/' + category)
    print(f'Extracting Data of Class: {category}, {i}')
    for name in category_files:
      data_list.append(np.load(os.path.join(DATASET_PATH, category, name)).reshape((ACTION_BANK_ROWS, ACTION_BANK_COLS)))
      labels_list.append(i)
  return data_list, labels_list

In [3]:
def create_2d_cnn_model():
  model = Sequential()

  model.add(Conv2D(filters=1, kernel_size=(1, 21), activation='tanh', use_bias=True, activity_regularizer=l1(0.001), data_format="channels_last",
                   input_shape=(ACTION_BANK_ROWS, ACTION_BANK_COLS, 1)))

  model.add(Conv2D(filters=1, kernel_size=(1, 2), activation='tanh', strides=(1, 2), data_format='channels_last'))

  model.add(Dropout(0.2))

  model.add(Conv2D(filters=2, kernel_size=(1, 21), activation='tanh', use_bias=True, activity_regularizer=l1(0.001), data_format='channels_last'))

  model.add(Conv2D(filters=2, kernel_size=(1, 2), activation='tanh', strides=(1, 2), data_format='channels_last'))

  model.add(Dropout(0.2))

  model.add(Flatten())

  model.add(Dense(len(ALL_CLASSES), activation="softmax"))
    
  print(model.summary())

  plot_model(model, to_file='2d_cnn_model.png', show_shapes = True, show_layer_names = True)

  return model

In [4]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name, x_label, y_label, save_name):
  metric_value_1 = model_training_history.history[metric_name_1]
  metric_value_2 = model_training_history.history[metric_name_2]

  epochs = range(len(metric_value_1))

  plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
  plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

  plt.title(str(plot_name))
  plt.legend()
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.savefig(save_name)

In [ ]:
data_list, labels_list = create_dataset()
labels_list = to_categorical(labels_list)

x_train, x_test, y_train, y_test = train_test_split(np.asarray(data_list), np.asarray(labels_list))
x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.int32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

model = create_2d_cnn_model()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.CategoricalAccuracy())
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
model_training_history = model.fit(x=x_train, y=y_train, epochs=50, batch_size=4, shuffle=True, validation_split=0.2, callbacks=[early_stopping_callback])

print('-------------------------Evaluating model now-------------------------')
model_testing_history = model.evaluate(x_test, y_test)

plot_metric(model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss', 'Epochs', 'Loss', '2d_cnn_loss_plot')

model_evaluation_loss, model_evaluation_accuracy = model_testing_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'conv2D_model_DateTime_{current_date_time_string}_Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
model.save(model_file_name)


In [ ]:
plot_metric(model_training_history, 'categorical_accuracy', 'val_categorical_accuracy', 'Total Accuracy vs Total Validation Accuracy',
            'Epochs', 'Accuracy', '2d_cnn_accuracy_plot')